# 0. Importing Libraries & Data

In [29]:
# Data Manipulation
import pandas as pd
import numpy as np

# Data Scrubbing
from sklearn.impute import SimpleImputer

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Modelling Tool
from xgboost import XGBClassifier

# Testing & Validation
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Misc
import warnings

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

This dataset can be found at https://www.kaggle.com/competitions/spaceship-titanic 
***
#### Data Fields:
__PassengerId__ <br>
A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.<br>
__HomePlanet__ <br>
The planet the passenger departed from, typically their planet of permanent residence.<br>
__CryoSleep__ <br>
Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.<br>
__Cabin__ <br>
The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.<br>
__Destination__ <br>
The planet the passenger will be debarking to.<br>
__Age__ <br>
The age of the passenger.<br>
__VIP__ <br>
Whether the passenger has paid for special VIP service during the voyage.<br>
__RoomService, FoodCourt, ShoppingMall, Spa, VRDeck__ <br>
Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.<br>
__Name__ <br>
The first and last names of the passenger.<br>
__Transported__ <br>
Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.<br>

In [3]:
#titanic_data = pd.read_csv("/home/mmmarinov/ProjectPortfolio/1. Titanic_SpaceShip - Binary Classification/Titanic_SpaceShip_Train_Data.csv")
titanic_data = pd.read_csv(r"C:\Users\N179960\OneDrive - Munich Re\Martin Stuff\Personal\ProjectPortfolio\1. Titanic_SpaceShip - Binary Classification\Titanic_SpaceShip_Train_Data.csv")

# 1. Investigate the Data

## 1.1 High-level understanding
This segment is to get a general sense of what kind of information is held

In [4]:
titanic_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [5]:
titanic_data.info()

# Looking at what data types the dataset hold. Which needs to be changeed to float, and which whill need to be One-Hot Encoded

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [6]:
titanic_data.describe(include='all')

# Viewing the mean, std, min, max for numerical values and unique for catagorical values
# Looking for an initial understanding of the spread and common values found in the dataset

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
count,8693,8492,8476,8494,8511,8514.000000,8490,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000,8493,8693
unique,8693,3,2,6560,3,NaN,2,NaN,NaN,NaN,NaN,NaN,8473,2
top,0001_01,Earth,False,G/734/S,TRAPPIST-1e,NaN,False,NaN,NaN,NaN,NaN,NaN,Gollux Reedall,True
freq,1,4602,5439,8,5915,NaN,8291,NaN,NaN,NaN,NaN,NaN,2,4378
mean,NaN,NaN,NaN,NaN,NaN,28.827930,NaN,224.687617,458.077203,173.729169,311.138778,304.854791,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,14.489021,NaN,666.717663,1611.489240,604.696458,1136.705535,1145.717189,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,19.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,27.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,38.000000,NaN,47.000000,76.000000,27.000000,59.000000,46.000000,NaN,NaN


In [14]:
titanic_data.isna().sum()

# Each column has missing information. Since it's numerical and catagorical, likely I'll need to use a mix of techniques to populate the values

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

Numerical: <br>
- Passenger_Id (If seperated) 
- Age (#)
- RoomService ($)
- FoodCourt ($)
- ShoppingMall ($)
- Spa ($)
- VRDeck ($)

Catagorical: <br>
- HomePlant (Unique: 3)
- Cryosleep (T/F)
- Cabin (Deck and Side)
- Destination (Unique: 3)
- VIP (T/F)
- Name (Mostly Unique)

Target Variable: <br>
- Transported (T/F)

## 1.2 Explore the Data & Analysis

In [7]:
# Make a copy so that the original remains untouched
titanic_data_eda = titanic_data.copy()

In [11]:
# Breaking up Cabin to understand if there are trends with deck level or side of ship
titanic_data_eda[['Deck','Cabin_num','Side']] = titanic_data_eda['Cabin'].str.split('/', expand = True)

# Breaking up Passenger to view grouping
titanic_data_eda[['Pass_group','Pass_id']] = titanic_data_eda['PassengerId'].str.split('_', expand = True)



In [18]:
titanic_data_eda_dummies = pd.get_dummies(titanic_data_eda, columns=['HomePlanet','Destination','Deck','Side'])

In [19]:
titanic_data_eda_dummies

,PassengerId,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Cabin_num,Pass_group,Pass_id,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Side_P,Side_S
0,0001_01,0.0,B/0/P,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0,0001,01,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0
1,0002_01,0.0,F/0/S,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,0,0002,01,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
2,0003_01,0.0,A/0/S,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0,0003,01,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1
3,0003_02,0.0,A/0/S,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0,0003,02,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1
4,0004_01,0.0,F/1/S,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,1,0004,01,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0.0,A/98/P,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,98,9276,01,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0
8689,9278_01,1.0,G/1499/S,18.0,0.0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,1499,9278,01,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1
8690,9279_01,0.0,G/1500/S,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,1500,9279,01,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
8691,9280_01,0.0,E/608/S,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,608,9280,01,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1


In [21]:
titanic_data_eda_dropped = titanic_data_eda_dummies.drop(['PassengerId','Cabin','Name','Cabin_num','Pass_id'],axis=1)
titanic_data_eda_dropped

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Pass_group,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Side_P,Side_S
0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0001,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0
1,0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,True,0002,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
2,0.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,False,0003,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1
3,0.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,False,0003,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1
4,0.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,True,0004,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,False,9276,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0
8689,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,False,9278,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1
8690,0.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,True,9279,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
8691,0.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,False,9280,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1


In [27]:
# Changing the catagorical fields into numerical so that it can all be analyzed
titanic_data_eda_dropped[['CryoSleep','VIP','Transported','Pass_group']] = titanic_data_eda_dropped[['CryoSleep','VIP','Transported','Pass_group']].astype('float64')

In [28]:
titanic_data_eda_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CryoSleep                  8476 non-null   float64
 1   Age                        8514 non-null   float64
 2   VIP                        8490 non-null   float64
 3   RoomService                8512 non-null   float64
 4   FoodCourt                  8510 non-null   float64
 5   ShoppingMall               8485 non-null   float64
 6   Spa                        8510 non-null   float64
 7   VRDeck                     8505 non-null   float64
 8   Transported                8693 non-null   float64
 9   Pass_group                 8693 non-null   float64
 10  HomePlanet_Earth           8693 non-null   uint8  
 11  HomePlanet_Europa          8693 non-null   uint8  
 12  HomePlanet_Mars            8693 non-null   uint8  
 13  Destination_55 Cancri e    8693 non-null   uint8

In [ ]:
print(titanic_data_eda_dropped.corr())

In [31]:
sns.pairplot(titanic_data_eda_dropped)